In [37]:
import os
import json
import tap
import wandb
import pandas as pd

target_to_method_name_dict = {
    'diffusion_policy.policy.diffuser_actor.DiffuserActor': 'baseline',
    'diffusion_policy.policy.diffuser_actor_pose_invariant_v2.DiffuserActor': 'ours',
    'diffusion_policy.policy.diffuser_actor_lowdim.DiffuserActor': 'baseline',
    'diffusion_policy.policy.diffuser_actor_pose_invariant_lowdim.DiffuserActor': 'pose invariant',
    'diffusion_policy.policy.diffuser_actor_pose_invariant_lowdim_v2.DiffuserActor': 'point invariant',
}

def target_to_method_name(target, mask):
    method = target_to_method_name_dict[target]
    if mask:
        method += " [mask]"
    return method

def get_method_name(run):
    use_mask = run.config['policy'].get('use_mask', False)
    method = target_to_method_name(run.config['policy']['_target_'], use_mask)
    return method


def get_experiment_state(ids, api, username, project_name):
    df = pd.DataFrame(columns=['method', 'task', 'id', 'epoch', 'train_success_rate', 'eval_success_rate', 'state', '%', 'runtime', 'expected_finish', 'node'])
    for id in ids:
        try:
            run = api.run(f"{username}/{project_name}/{id}")     
            id = run.id
            task = run.config['task']['task_name']
            epoch = run.summary.get('epoch', None)
            train_success_rate = run.summary.get('train_success_rate', None)
            val_success_rate = run.summary.get('eval_success_rate', None)
            method = get_method_name(run)
            percent = min((epoch + 1) / run.config['training']['num_epochs'] * 100, 100)
            runtime = run.summary.get('_runtime', None)
            if runtime is not None:
                runtime = runtime / 60 / 60 / 24
                if percent >= 100:
                    expected_finish = "Done"
                else:
                    expected_finish = (100 - percent) / percent  * runtime
                    expected_finish = f"{expected_finish:.2f} days"
                runtime = f"{runtime:.2f} days"
            else:
                expected_finish = None

            if train_success_rate is not None:
                state = "✅"
            else:
                if percent == 100:
                    state = "🤖"
                else:
                    state = "🔥"
        
            url = f'https://wandb.ai/{username}/{project_name}/runs/{id}'
            id = '<a target="_blank" href="{}">{}</a>'.format(url, id)

            if 'felix' in run.tags:
                node = 'felix'
            elif 'basti' in run.tags:
                node = 'basti'
            elif 'julen' in run.tags:
                node = 'julen'
            else:
                node = 'unknown'

            # df.loc[len(df)] = [id, epoch, train_success_rate, val_success_rate]
            df = pd.concat([df, pd.DataFrame([[method, task, id, epoch, train_success_rate, val_success_rate, state, percent, runtime, expected_finish, node]], columns=df.columns)], ignore_index=True)
        except Exception as e:
            print(f"Error: {e}")
            print(f"{id}")

    def make_clickable(id):
        # target _blank to open new window
        url = f'https://wandb.ai/{username}/{project_name}/runs/{id}'
        return '<a target="_blank" href="{}">{}</a>'.format(url, id)

    # df.style.format({'id': make_clickable})

    return df

ids_path_highdim = os.path.join(os.environ['DIFFUSION_POLICY_ROOT'], 'tools/highdim_wandb_ids.txt')
ids_path_lowdim = os.path.join(os.environ['DIFFUSION_POLICY_ROOT'], 'tools/lowdim_wandb_ids.txt')
username = 'felix-herrmann'
project_name = 'diffusion_policy_debug'

wandb.login()
api = wandb.Api()




In [38]:
from IPython.core.display import display, HTML

with open(ids_path_highdim, 'r') as file:
    content = file.read()
    lines = content.splitlines()
    ids = [line.split(' ')[0] for line in lines]

df = get_experiment_state(ids, api, username, project_name)
HTML(df.to_html(render_links=True, escape=False))


/tmp/ipykernel_146251/4230423001.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


,method,task,id,epoch,train_success_rate,eval_success_rate,state,%,runtime,expected_finish,node
0,ours [mask],open_drawer,1pe5oi9p,4999,None,None,🤖,100.0,0.43 days,Done,julen
1,ours [mask],put_item_in_drawer,gav4tk3g,4969,None,None,🔥,99.4,1.82 days,0.01 days,felix
2,ours [mask],stack_blocks,rjdc7f2g,4999,None,None,🤖,100.0,1.15 days,Done,julen
3,ours [mask],sweep_to_dustpan_of_size,f3bpz3fg,4999,None,None,🤖,100.0,0.76 days,Done,julen
4,ours [mask],turn_tap,97bpb1is,4999,None,None,🤖,100.0,0.22 days,Done,julen
5,baseline,open_drawer,wrgldwzy,4999,None,None,🤖,100.0,1.68 days,Done,basti
6,baseline,put_item_in_drawer,g3aswx29,1345,None,None,🔥,26.92,1.80 days,4.89 days,basti
7,baseline,stack_blocks,1o6uz7dc,1451,None,None,🔥,29.04,1.80 days,4.40 days,basti
8,baseline,turn_tap,s03igxyv,651,None,None,🔥,13.04,0.10 days,0.69 days,julen
9,baseline,sweep_to_dustpan_of_size,rjpzbtd2,281,None,None,🔥,5.64,0.10 days,1.74 days,julen


In [39]:
with open(ids_path_lowdim, 'r') as file:
    content = file.read()
    lines = content.splitlines()
    ids = [line.split(' ')[0] for line in lines]

df = get_experiment_state(ids, api, username, project_name)
HTML(df.to_html(render_links=True, escape=False))



,method,task,id,epoch,train_success_rate,eval_success_rate,state,%,runtime,expected_finish,node
0,baseline,open_drawer,"<a target=""_blank"" href=""https://wandb.ai/feli...",2000,0.88,0.84,✅,100,0.29 days,Done,julen
1,baseline,put_item_in_drawer,"<a target=""_blank"" href=""https://wandb.ai/feli...",1999,None,None,🤖,100.0,1.04 days,Done,julen
2,baseline,stack_blocks,"<a target=""_blank"" href=""https://wandb.ai/feli...",1999,None,None,🤖,100.0,1.02 days,Done,julen
3,baseline,sweep_to_dustpan_of_size,"<a target=""_blank"" href=""https://wandb.ai/feli...",2000,0.96,0.96,✅,100,0.50 days,Done,julen
4,baseline,turn_tap,"<a target=""_blank"" href=""https://wandb.ai/feli...",2000,0.8,0.84,✅,100,0.11 days,Done,julen
5,pose invariant,open_drawer,"<a target=""_blank"" href=""https://wandb.ai/feli...",2000,0.64,0.48,✅,100,0.28 days,Done,julen
6,pose invariant,put_item_in_drawer,"<a target=""_blank"" href=""https://wandb.ai/feli...",1999,None,None,🤖,100.0,1.01 days,Done,julen
7,pose invariant,stack_blocks,"<a target=""_blank"" href=""https://wandb.ai/feli...",1999,None,None,🤖,100.0,1.00 days,Done,julen
8,pose invariant,sweep_to_dustpan_of_size,"<a target=""_blank"" href=""https://wandb.ai/feli...",2000,0.68,0.68,✅,100,0.47 days,Done,julen
9,pose invariant,turn_tap,"<a target=""_blank"" href=""https://wandb.ai/feli...",2000,0.44,0.4,✅,100,0.14 days,Done,julen
